## Density Profiler Visualization:
1) Set up the parameters

2) Load and Plot interactive Graph

In [1]:
from seeq import spy
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

%matplotlib inline 
# %matplotlib notebook

from ipywidgets import interactive, HBox, VBox, interact, Layout
import ipywidgets as wg

from IPython.display import clear_output

import datetime

In [31]:
userName = wg.Text(
    value='user@bp.com',
    placeholder='Type Username',
    description='Enter BP mail ',
    disabled=False,
style = {'description_width': 'initial'}
)
password = wg.Text(
    value='Password',
    placeholder='Type Password',
    description='Enter BP Password ',
    disabled=False,
style = {'description_width': 'initial'}
)

log = wg.HBox([userName,password])
display(log)

In [2]:
#Date Picker widgets
style = {'description_width': 'initial'}

startTime = wg.DatePicker(
    value = datetime.date.today() - datetime.timedelta(days=1),
    description='Starting date',
    disabled=False,
    style=style
)
startTimeh = wg.Text(
    value='01:00:00',
    placeholder='Type time',
    description='HH:MM:SS ',
    disabled=False
)

endTime = wg.DatePicker(
    value = datetime.date.today(),
    description='Ending date',
    disabled=False,
    style=style
)
endTimeh = wg.Text(
    value='00:00:00',
    placeholder='Type time',
    description='HH:MM:SS ',
    disabled=False
)

asset = wg.Dropdown(
    options=[('IP Sep', 'PSVM-1A-DI022**.PV PSVM-1A-XI02205**.PV'), ('Treater', 'PSVM-1A-DI023**.PV PSVM-1A-XI0230**.PV'),
             ('Desalter', 'PSVM-1A-DI024**.PV PSVM-1A-XI0240**.PV'), ('HP Sep A', 'PSVM-1A-DI02105**.PV PSVM-1A-XI02205**.PV')],
    value='PSVM-1A-DI022**.PV PSVM-1A-XI02205**.PV',
    description='Asset:',
)
fequency = wg.Dropdown(
    options=[('1Min', '1min'),('5Min','5min'),('15Min','15min'),('30Min','30min'),('60Min','60min')],
    value='1min',
    description='Frequency:',
)
v1 = wg.VBox([startTime,startTimeh])
v2 = wg.VBox([endTime,endTimeh])
v3 = wg.VBox([asset,fequency])
l = wg.HBox([v1,v2, v3])
display(l)

In [7]:
def getLogin():
    f = open("credentials.key", "w")   # 'r' for reading and 'w' for writing
    f.write("{} \n{}".format(userName.value,password.value))    # Write inside file 
    f.close()  
    
def getData(StartTime, EndTime, Freq, Sep, SepLimits):
    my_items = pd.DataFrame({'Name': [Sep]})
    x = spy.search(my_items)
    my_data = spy.pull(x, start=StartTime, end=EndTime, grid=Freq, header='Name')
    my_data = my_data.reindex(sorted(my_data.columns), axis=1)
    # my_data

    my_new_items = pd.DataFrame({'Name': [SepLimits]})
    x_limits = spy.search(my_new_items)

    my_data_limits = spy.pull(x_limits, start=StartTime, end=EndTime, grid=Freq, header='Name')
    # my_data = my_data.reindex(sorted(my_data.columns), axis=1)
    # my_data_limits
    return my_data, my_data_limits

def plotData(b):

    sep1 = asset.value.split()[0]
    sepLimits1 = asset.value.split()[1]
    startTime1 = startTime.value.strftime("%D") + " " + startTimeh.value
    endTime1 = endTime.value.strftime("%D") + " " + endTimeh.value
    freq = fequency.value#'60min'
    my_data, my_data_limits = getData(startTime1, endTime1, freq, sep1, sepLimits1)
    x = (np.arange(len(my_data.columns), 0, -1)/len(my_data.columns))*100

    clear_output(wait=True)
    slider = wg.IntSlider(value=1,min=1,max=len(my_data)-1,step=1,description='Date',layout={'width': '800px'})
    play = wg.Play(value=1,min=1,max=len(my_data)-1,step=1,interval=1211,description="Press play",disabled=False)
    g = wg.jslink((play, 'value'), (slider, 'value'))

    @interact
    def plotF(a = slider,b=play):
        plt.rcParams['figure.figsize'] = (20,8)
        plt.rcParams.update({'font.size': 18})
        plt.rcParams['axes.xmargin'] = 0
        plt.rcParams['axes.ymargin'] = 0

        y = my_data.iloc[b]
        limits = my_data_limits.iloc[b].sort_values()

        Gas = y <= limits[0]
        Foam = (y > limits[0]) & (y <= limits[1])
        Oil = (y > limits[1]) & (y <= limits[2])
        Emul = (y > limits[2]) & (y <= limits[3])
        Water = (y > limits[3]) & (y <= limits[4])
        Sand = y > limits[4]

        x1 = plt.barh(x[Gas],y[Gas],color = 'gray', edgecolor='black', label='Gas')
        x2 = plt.barh(x[Foam],y[Foam],color = 'yellow', edgecolor='black', label='Foam')
        x3 = plt.barh(x[Oil],y[Oil],color = 'black', edgecolor='black', label='Oil')
        x4 = plt.barh(x[Emul],y[Emul],color = 'orange', edgecolor='black', label='Emul')
        x5 = plt.barh(x[Water],y[Water],color = 'blue', edgecolor='black', label='Water')
        x6 = plt.barh(x[Sand],y[Sand],color = 'red', edgecolor='black', label='Sand')


        gas = mpatches.Patch(color='gray', label='Gas')
        foam = mpatches.Patch(color='yellow', label='Foam')
        oil = mpatches.Patch(color='black', label='Oil')
        emul = mpatches.Patch(color='orange', label='Emul')
        water = mpatches.Patch(color='blue', label='Water')
        sand = mpatches.Patch(color='red', label='Sand')
        plt.legend(handles=[gas,foam,oil,emul,water,sand])
#         print(my_data.index[b].strftime("%b %d %Y %H:%M:%S"))
        plt.title('DP view for separator at: {}'.format(my_data.index[b].strftime("%b %d %Y %H:%M:%S")))
        plt.show()
    

        
# x,y = getData(startTime, endTime, freq, sep, sepLimits)
from IPython.display import display
button = wg.Button(description="Load & Plot Data")

output = wg.Output()
display(button,output)


def on_button_clicked(b):

    with output:
        print("Button clicked.")
        getLogin()
        spy.login(url='https://seeq-eh.bpweb.bp.com', credentials_file = 'credentials.key', directory='Minimal AD configuration', ignore_ssl_errors=True)
        clear_output(wait=True)
        plotData(b)

button.on_click(on_button_clicked)



Button(description='Load & Plot Data', style=ButtonStyle())

Output()